In [38]:
from API_utils import *
from dask import delayed, compute
from dask.diagnostics import ProgressBar
import subprocess
import os
import json
from dataclasses import dataclass


In [6]:
site = 'TEAK'
productcode = 'DP1.30003.001'
data_path = '/home/jovyan/tmp'
date = '2018-06'
os.makedirs(data_path, exist_ok=True)

t0, laz = generate_laz_download_info(productcode, site, date)

results = []
for f in laz:
    results.append(delayed(download_from_NEON_API)(f, data_path))
    
with ProgressBar():
    computed = compute(*results)

In [22]:

f = os.path.join(data_path, laz[0]['name'])

cmd = f'pdal info {f}'
reply = subprocess.run(cmd, shell=True, capture_output=True)
meta = json.loads(reply.stdout)
bbox = meta['stats']['bbox']['native']['bbox']
bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']])

In [32]:
bbox = meta['stats']['bbox']['native']['bbox']
bounds = ([bbox['minx'], bbox['maxx']], [bbox['miny'], bbox['maxy']])

In [47]:
meta

{'file_size': 53867890,
 'filename': '/home/jovyan/tmp/NEON_D17_TEAK_DP1_320000_4104000_classified_point_cloud_colorized.laz',
 'now': '2021-02-15T23:11:17+0000',
 'pdal_version': '2.2.0 (git-version: 47145b)',
 'reader': 'readers.las',
 'stats': {'bbox': {'EPSG:4326': {'bbox': {'maxx': -119.0133828,
     'maxy': 37.07419704,
     'maxz': 3666.787,
     'minx': -119.0248639,
     'miny': 37.06499689,
     'minz': 1655.367},
    'boundary': {'type': 'Polygon',
     'coordinates': [[[-119.02462432721693, 37.064996894266606, 1655.367],
       [-119.02486390208087, 37.074005565812065, 1655.367],
       [-119.01362101426926, 37.07419704274282, 3666.7870000000003],
       [-119.01338276742766, 37.065188309000625, 3666.7870000000003],
       [-119.02462432721693, 37.064996894266606, 1655.367]]]}},
   'native': {'bbox': {'maxx': 320999.999,
     'maxy': 4104999.999,
     'maxz': 3666.787,
     'minx': 320000,
     'miny': 4104000,
     'minz': 1655.367},
    'boundary': {'type': 'Polygon',
   

In [39]:
@dataclass
class las_tile_index(dict):
    '''Indexes las or laz tiles for easy retireval of points'''
    
    def fetch(loc, window):
        pass

The local coordinate system for the laz files at TEAK are UTM11  (this was not easy to determine, I found it here https://www.neonscience.org/field-sites/teak).

In [42]:
cmd = f'entwine build -i {data_path}/*.laz -o ept_phili --srs "EPSG:32651"'

In [43]:
cmd

'entwine build -i /home/jovyan/tmp/*.laz -o ept_phili --srs "EPSG:32651"'